# Facial Key-Point Detection 

Kaggle Challenge: Facial Keypoints Detection
https://www.kaggle.com/c/facial-keypoints-detection/overview

Objective: Train a Convolutional Neural Net to detect Facial Keypoints in images 

In [36]:
#import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras import optimizers
from sklearn.utils import shuffle 
from sklearn.model_selection import train_test_split
from keras.models import Sequential, load_model 
from keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense



In [2]:
#load the training dataset 
df = pd.read_csv('facial key-point data/training.csv')

In [3]:
df.head()

,left_eye_center_x,left_eye_center_y,right_eye_center_x,right_eye_center_y,left_eye_inner_corner_x,left_eye_inner_corner_y,left_eye_outer_corner_x,left_eye_outer_corner_y,right_eye_inner_corner_x,right_eye_inner_corner_y,...,nose_tip_y,mouth_left_corner_x,mouth_left_corner_y,mouth_right_corner_x,mouth_right_corner_y,mouth_center_top_lip_x,mouth_center_top_lip_y,mouth_center_bottom_lip_x,mouth_center_bottom_lip_y,Image
0,66.033564,39.002274,30.227008,36.421678,59.582075,39.647423,73.130346,39.969997,36.356571,37.389402,...,57.066803,61.195308,79.970165,28.614496,77.388992,43.312602,72.935459,43.130707,84.485774,238 236 237 238 240 240 239 241 241 243 240 23...
1,64.332936,34.970077,29.949277,33.448715,58.856170,35.274349,70.722723,36.187166,36.034723,34.361532,...,55.660936,56.421447,76.352000,35.122383,76.047660,46.684596,70.266553,45.467915,85.480170,219 215 204 196 204 211 212 200 180 168 178 19...
2,65.057053,34.909642,30.903789,34.909642,59.412000,36.320968,70.984421,36.320968,37.678105,36.320968,...,53.538947,60.822947,73.014316,33.726316,72.732000,47.274947,70.191789,47.274947,78.659368,144 142 159 180 188 188 184 180 167 132 84 59 ...
3,65.225739,37.261774,32.023096,37.261774,60.003339,39.127179,72.314713,38.380967,37.618643,38.754115,...,54.166539,65.598887,72.703722,37.245496,74.195478,50.303165,70.091687,51.561183,78.268383,193 192 193 194 194 194 193 192 168 111 50 12 ...
4,66.725301,39.621261,32.244810,38.042032,58.565890,39.621261,72.515926,39.884466,36.982380,39.094852,...,64.889521,60.671411,77.523239,31.191755,76.997301,44.962748,73.707387,44.227141,86.871166,147 148 160 196 215 214 216 217 219 220 206 18...


In [4]:
#The image column contains space seperated pixel values
#Changing the above to a numpy array

df['Image'] = df['Image'].apply(lambda x: np.fromstring(x, sep=' '))
df['Image'].head()
df['Image']





0       [238.0, 236.0, 237.0, 238.0, 240.0, 240.0, 239...
1       [219.0, 215.0, 204.0, 196.0, 204.0, 211.0, 212...
2       [144.0, 142.0, 159.0, 180.0, 188.0, 188.0, 184...
3       [193.0, 192.0, 193.0, 194.0, 194.0, 194.0, 193...
4       [147.0, 148.0, 160.0, 196.0, 215.0, 214.0, 216...
5       [167.0, 169.0, 170.0, 167.0, 156.0, 145.0, 106...
6       [109.0, 109.0, 125.0, 141.0, 145.0, 139.0, 120...
7       [178.0, 177.0, 178.0, 179.0, 179.0, 179.0, 181...
8       [164.0, 158.0, 118.0, 76.0, 66.0, 69.0, 59.0, ...
9       [226.0, 227.0, 225.0, 224.0, 221.0, 220.0, 215...
10      [52.0, 51.0, 54.0, 57.0, 57.0, 56.0, 55.0, 55....
11      [142.0, 124.0, 123.0, 133.0, 140.0, 147.0, 151...
12      [86.0, 93.0, 106.0, 114.0, 112.0, 110.0, 111.0...
13      [75.0, 65.0, 63.0, 75.0, 70.0, 78.0, 84.0, 97....
14      [119.0, 106.0, 101.0, 106.0, 89.0, 66.0, 67.0,...
15      [52.0, 54.0, 74.0, 76.0, 55.0, 48.0, 47.0, 41....
16      [201.0, 200.0, 202.0, 193.0, 124.0, 59.0, 53.0...
17      [202.0

In [5]:
len(df['Image'][0])

9216

In [6]:
'''Checking if the dataframe has any rows 
with null values and then removing those 
rows from the dataframe
'''
print(df.isnull().sum())
df = df.dropna(how='any',axis=0)
print("After removing the null values")
print(df.isnull().sum())


left_eye_center_x              10
left_eye_center_y              10
right_eye_center_x             13
right_eye_center_y             13
left_eye_inner_corner_x      4778
left_eye_inner_corner_y      4778
left_eye_outer_corner_x      4782
left_eye_outer_corner_y      4782
right_eye_inner_corner_x     4781
right_eye_inner_corner_y     4781
right_eye_outer_corner_x     4781
right_eye_outer_corner_y     4781
left_eyebrow_inner_end_x     4779
left_eyebrow_inner_end_y     4779
left_eyebrow_outer_end_x     4824
left_eyebrow_outer_end_y     4824
right_eyebrow_inner_end_x    4779
right_eyebrow_inner_end_y    4779
right_eyebrow_outer_end_x    4813
right_eyebrow_outer_end_y    4813
nose_tip_x                      0
nose_tip_y                      0
mouth_left_corner_x          4780
mouth_left_corner_y          4780
mouth_right_corner_x         4779
mouth_right_corner_y         4779
mouth_center_top_lip_x       4774
mouth_center_top_lip_y       4774
mouth_center_bottom_lip_x      33
mouth_center_b

In [7]:
#Splitting the training dataset into X_tr(features: image pixels) and Y_tr(facial key-points)
#The Image column contains pixel values ranging from 0-255. These can be normalized to range from 0 to 1

X_tr = (np.vstack(df['Image'].values)/255).astype(float)
#The images can be rescaled to 96X96X1
#Downscaling the image helps with faster processing 
X_tr = X_tr.reshape(-1,96,96,1)
#Setting Y_tr as the facial keypoints and normalizing them to range from -1 to 1 
Y_tr = ((df[df.columns[:-1]].values)-48)/48
Y_tr = Y_tr.astype(float)
#print(Y_tr)



### Building the Model

In [30]:
#function to build the convolutional neural network model
def build_model():
    model = Sequential()
    model.add(Convolution2D(32,(5,5), input_shape=(96,96,1),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    
    model.add(Convolution2D(64,(3,3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.1))
    
    
    model.add(Convolution2D(128,(3,3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
   
    
    model.add(Convolution2D(64,(3,3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.3))
    
    
    model.add(Convolution2D(32,(3,3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    
    
    model.add(Flatten())
    model.add(Dense(30))
    
    return model
    
    
    
    
    

In [31]:
model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_39 (Conv2D)           (None, 92, 92, 32)        832       
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 46, 46, 32)        0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 44, 44, 64)        18496     
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 20, 20, 128)       73856     
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 10, 10, 128)       0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 8, 8, 64)          73792     
__________

In [38]:
#Compiling and training the model with RMSProp Optimizer
rmsprop = optimizers.RMSprop()
model.compile(optimizer=rmsprop,loss='mean_squared_error',metrics=['accuracy'])



In [39]:

model.fit(X_tr,Y_tr,epochs=500,batch_size=200)

Epoch 1/500
2140/2140 [==============================] - 16s 8ms/step - loss: 0.1808 - acc: 0.6220
Epoch 2/500
2140/2140 [==============================] - 16s 7ms/step - loss: 0.0043 - acc: 0.7051
Epoch 3/500
2140/2140 [==============================] - 16s 7ms/step - loss: 0.0043 - acc: 0.7051
Epoch 4/500
2140/2140 [==============================] - 16s 7ms/step - loss: 0.0044 - acc: 0.7051
Epoch 5/500
2140/2140 [==============================] - 16s 8ms/step - loss: 0.0049 - acc: 0.7051
Epoch 6/500
2140/2140 [==============================] - 16s 7ms/step - loss: 0.0079 - acc: 0.7051
Epoch 7/500
2140/2140 [==============================] - 18s 9ms/step - loss: 0.0082 - acc: 0.7051
Epoch 8/500
2140/2140 [==============================] - 17s 8ms/step - loss: 0.0079 - acc: 0.7051
Epoch 9/500
2140/2140 [==============================] - 17s 8ms/step - loss: 0.0071 - acc: 0.7051
Epoch 10/500
2140/2140 [==============================] - 17s 8ms/step - loss: 0.0071 - acc: 0.7051
Epoch 11/

2140/2140 [==============================] - 16s 8ms/step - loss: 0.0026 - acc: 0.7192
Epoch 84/500
2140/2140 [==============================] - 16s 7ms/step - loss: 0.0033 - acc: 0.7168
Epoch 85/500
2140/2140 [==============================] - 16s 7ms/step - loss: 0.0022 - acc: 0.7159
Epoch 86/500
2140/2140 [==============================] - 16s 7ms/step - loss: 0.0038 - acc: 0.7178
Epoch 87/500
2140/2140 [==============================] - 16s 7ms/step - loss: 0.0022 - acc: 0.7220
Epoch 88/500
2140/2140 [==============================] - 16s 7ms/step - loss: 0.0042 - acc: 0.7103
Epoch 89/500
2140/2140 [==============================] - 16s 7ms/step - loss: 0.0021 - acc: 0.7215
Epoch 90/500
2140/2140 [==============================] - 16s 8ms/step - loss: 0.0024 - acc: 0.7248
Epoch 91/500
2140/2140 [==============================] - 16s 7ms/step - loss: 0.0042 - acc: 0.7262
Epoch 92/500
2140/2140 [==============================] - 16s 7ms/step - loss: 0.0021 - acc: 0.7294
Epoch 93/500


2140/2140 [==============================] - 18s 9ms/step - loss: 0.0010 - acc: 0.7864
Epoch 165/500
2140/2140 [==============================] - 19s 9ms/step - loss: 0.0011 - acc: 0.7911
Epoch 166/500
2140/2140 [==============================] - 18s 8ms/step - loss: 0.0010 - acc: 0.7958
Epoch 167/500
2140/2140 [==============================] - 19s 9ms/step - loss: 0.0012 - acc: 0.7850
Epoch 168/500
2140/2140 [==============================] - 18s 8ms/step - loss: 9.1037e-04 - acc: 0.7981
Epoch 169/500
2140/2140 [==============================] - 19s 9ms/step - loss: 9.1903e-04 - acc: 0.8028
Epoch 170/500
2140/2140 [==============================] - 19s 9ms/step - loss: 0.0011 - acc: 0.7986
Epoch 171/500
2140/2140 [==============================] - 19s 9ms/step - loss: 9.7673e-04 - acc: 0.7963
Epoch 172/500
2140/2140 [==============================] - 18s 8ms/step - loss: 0.0011 - acc: 0.7860
Epoch 173/500
2140/2140 [==============================] - 18s 9ms/step - loss: 8.0912e-04 - 

2140/2140 [==============================] - 19s 9ms/step - loss: 5.9331e-04 - acc: 0.8439
Epoch 243/500
2140/2140 [==============================] - 18s 8ms/step - loss: 5.8011e-04 - acc: 0.8570
Epoch 244/500
2140/2140 [==============================] - 17s 8ms/step - loss: 4.9870e-04 - acc: 0.8505
Epoch 245/500
2140/2140 [==============================] - 18s 8ms/step - loss: 5.6394e-04 - acc: 0.8579
Epoch 246/500
2140/2140 [==============================] - 17s 8ms/step - loss: 6.6344e-04 - acc: 0.8379
Epoch 247/500
2140/2140 [==============================] - 18s 8ms/step - loss: 4.8868e-04 - acc: 0.8631
Epoch 248/500
2140/2140 [==============================] - 18s 8ms/step - loss: 5.5515e-04 - acc: 0.8472
Epoch 249/500
2140/2140 [==============================] - 17s 8ms/step - loss: 5.5395e-04 - acc: 0.8593
Epoch 250/500
2140/2140 [==============================] - 18s 8ms/step - loss: 5.6314e-04 - acc: 0.8505
Epoch 251/500
2140/2140 [==============================] - 17s 8ms/st

2140/2140 [==============================] - 19s 9ms/step - loss: 3.7478e-04 - acc: 0.8874
Epoch 321/500
2140/2140 [==============================] - 18s 8ms/step - loss: 2.9799e-04 - acc: 0.8893
Epoch 322/500
2140/2140 [==============================] - 19s 9ms/step - loss: 3.7328e-04 - acc: 0.8963
Epoch 323/500
2140/2140 [==============================] - 18s 8ms/step - loss: 3.7254e-04 - acc: 0.8949
Epoch 324/500
2140/2140 [==============================] - 17s 8ms/step - loss: 3.3312e-04 - acc: 0.8785
Epoch 325/500
2140/2140 [==============================] - 18s 8ms/step - loss: 4.3144e-04 - acc: 0.8836
Epoch 326/500
2140/2140 [==============================] - 17s 8ms/step - loss: 3.3669e-04 - acc: 0.8949
Epoch 327/500
2140/2140 [==============================] - 17s 8ms/step - loss: 3.3838e-04 - acc: 0.8921
Epoch 328/500
2140/2140 [==============================] - 18s 9ms/step - loss: 3.5649e-04 - acc: 0.8836
Epoch 329/500
2140/2140 [==============================] - 17s 8ms/st

2140/2140 [==============================] - 19s 9ms/step - loss: 3.2258e-04 - acc: 0.9065
Epoch 399/500
2140/2140 [==============================] - 18s 9ms/step - loss: 2.2491e-04 - acc: 0.8972
Epoch 400/500
2140/2140 [==============================] - 19s 9ms/step - loss: 3.0812e-04 - acc: 0.8981
Epoch 401/500
2140/2140 [==============================] - 19s 9ms/step - loss: 2.3657e-04 - acc: 0.9098
Epoch 402/500
2140/2140 [==============================] - 19s 9ms/step - loss: 2.9767e-04 - acc: 0.9121
Epoch 403/500
2140/2140 [==============================] - 18s 9ms/step - loss: 2.5269e-04 - acc: 0.9037
Epoch 404/500
2140/2140 [==============================] - 19s 9ms/step - loss: 2.5314e-04 - acc: 0.9173
Epoch 405/500
2140/2140 [==============================] - 19s 9ms/step - loss: 2.4797e-04 - acc: 0.9014
Epoch 406/500
2140/2140 [==============================] - 18s 8ms/step - loss: 2.9104e-04 - acc: 0.8981
Epoch 407/500
2140/2140 [==============================] - 18s 8ms/st

2140/2140 [==============================] - 19s 9ms/step - loss: 1.9656e-04 - acc: 0.9098
Epoch 477/500
2140/2140 [==============================] - 19s 9ms/step - loss: 2.2118e-04 - acc: 0.9140
Epoch 478/500
2140/2140 [==============================] - 20s 9ms/step - loss: 2.6436e-04 - acc: 0.9047
Epoch 479/500
2140/2140 [==============================] - 19s 9ms/step - loss: 1.9263e-04 - acc: 0.9215
Epoch 480/500
2140/2140 [==============================] - 20s 9ms/step - loss: 2.1863e-04 - acc: 0.9131
Epoch 481/500
2140/2140 [==============================] - 19s 9ms/step - loss: 1.8956e-04 - acc: 0.9192
Epoch 482/500
2140/2140 [==============================] - 18s 9ms/step - loss: 2.7303e-04 - acc: 0.9098
Epoch 483/500
2140/2140 [==============================] - 18s 8ms/step - loss: 2.0444e-04 - acc: 0.9140
Epoch 484/500
2140/2140 [==============================] - 19s 9ms/step - loss: 1.7210e-04 - acc: 0.9051
Epoch 485/500
2140/2140 [==============================] - 18s 9ms/st

In [42]:
#Compiling and training the model with SGD optimizer
#Training and validation split : 75%-25%
opt = optimizers.SGD(lr=0.01,momentum =0.9)
model.compile(optimizer=opt,loss='mean_squared_logarithmic_error',metrics=['mse','accuracy'])
model.fit(X_tr,Y_tr,epochs=500,batch_size=100,validation_split=0.25)

Train on 1605 samples, validate on 535 samples
Epoch 1/500
1605/1605 [==============================] - 15s 9ms/step - loss: 1.7999e-05 - mean_squared_error: 9.4024e-05 - acc: 0.9639 - val_loss: 3.3043e-05 - val_mean_squared_error: 1.5674e-04 - val_acc: 0.9009
Epoch 2/500
1605/1605 [==============================] - 15s 9ms/step - loss: 1.7996e-05 - mean_squared_error: 9.4001e-05 - acc: 0.9639 - val_loss: 3.3043e-05 - val_mean_squared_error: 1.5673e-04 - val_acc: 0.9009
Epoch 3/500
1605/1605 [==============================] - 15s 9ms/step - loss: 1.7992e-05 - mean_squared_error: 9.3989e-05 - acc: 0.9639 - val_loss: 3.3043e-05 - val_mean_squared_error: 1.5673e-04 - val_acc: 0.9009
Epoch 4/500
1605/1605 [==============================] - 14s 9ms/step - loss: 1.7988e-05 - mean_squared_error: 9.3964e-05 - acc: 0.9639 - val_loss: 3.3040e-05 - val_mean_squared_error: 1.5669e-04 - val_acc: 0.9009
Epoch 5/500
1605/1605 [==============================] - 16s 10ms/step - loss: 1.7984e-05 - mean_

Epoch 39/500
1605/1605 [==============================] - 16s 10ms/step - loss: 1.7852e-05 - mean_squared_error: 9.3150e-05 - acc: 0.9651 - val_loss: 3.2951e-05 - val_mean_squared_error: 1.5593e-04 - val_acc: 0.8991
Epoch 40/500
1605/1605 [==============================] - 18s 11ms/step - loss: 1.7847e-05 - mean_squared_error: 9.3118e-05 - acc: 0.9651 - val_loss: 3.2948e-05 - val_mean_squared_error: 1.5590e-04 - val_acc: 0.8991
Epoch 41/500
1605/1605 [==============================] - 15s 9ms/step - loss: 1.7844e-05 - mean_squared_error: 9.3086e-05 - acc: 0.9651 - val_loss: 3.2936e-05 - val_mean_squared_error: 1.5586e-04 - val_acc: 0.8991
Epoch 42/500
1605/1605 [==============================] - 15s 9ms/step - loss: 1.7840e-05 - mean_squared_error: 9.3057e-05 - acc: 0.9645 - val_loss: 3.2931e-05 - val_mean_squared_error: 1.5582e-04 - val_acc: 0.8972
Epoch 43/500
1605/1605 [==============================] - 17s 10ms/step - loss: 1.7837e-05 - mean_squared_error: 9.3043e-05 - acc: 0.9645 

1605/1605 [==============================] - 14s 9ms/step - loss: 1.7722e-05 - mean_squared_error: 9.2418e-05 - acc: 0.9651 - val_loss: 3.2848e-05 - val_mean_squared_error: 1.5519e-04 - val_acc: 0.8991
Epoch 78/500
1605/1605 [==============================] - 14s 9ms/step - loss: 1.7718e-05 - mean_squared_error: 9.2401e-05 - acc: 0.9645 - val_loss: 3.2846e-05 - val_mean_squared_error: 1.5518e-04 - val_acc: 0.8972
Epoch 79/500
1605/1605 [==============================] - 14s 8ms/step - loss: 1.7715e-05 - mean_squared_error: 9.2376e-05 - acc: 0.9645 - val_loss: 3.2839e-05 - val_mean_squared_error: 1.5516e-04 - val_acc: 0.8972
Epoch 80/500
1605/1605 [==============================] - 15s 9ms/step - loss: 1.7712e-05 - mean_squared_error: 9.2353e-05 - acc: 0.9645 - val_loss: 3.2835e-05 - val_mean_squared_error: 1.5512e-04 - val_acc: 0.8972
Epoch 81/500
1605/1605 [==============================] - 15s 9ms/step - loss: 1.7709e-05 - mean_squared_error: 9.2326e-05 - acc: 0.9651 - val_loss: 3.28

1605/1605 [==============================] - 14s 9ms/step - loss: 1.7610e-05 - mean_squared_error: 9.1817e-05 - acc: 0.9651 - val_loss: 3.2771e-05 - val_mean_squared_error: 1.5462e-04 - val_acc: 0.8991
Epoch 116/500
1605/1605 [==============================] - 15s 9ms/step - loss: 1.7606e-05 - mean_squared_error: 9.1804e-05 - acc: 0.9657 - val_loss: 3.2771e-05 - val_mean_squared_error: 1.5461e-04 - val_acc: 0.8991
Epoch 117/500
1605/1605 [==============================] - 15s 9ms/step - loss: 1.7602e-05 - mean_squared_error: 9.1798e-05 - acc: 0.9664 - val_loss: 3.2775e-05 - val_mean_squared_error: 1.5461e-04 - val_acc: 0.8991
Epoch 118/500
1605/1605 [==============================] - 15s 9ms/step - loss: 1.7600e-05 - mean_squared_error: 9.1790e-05 - acc: 0.9651 - val_loss: 3.2780e-05 - val_mean_squared_error: 1.5460e-04 - val_acc: 0.8991
Epoch 119/500
1605/1605 [==============================] - 16s 10ms/step - loss: 1.7597e-05 - mean_squared_error: 9.1773e-05 - acc: 0.9645 - val_loss:

Epoch 153/500
1605/1605 [==============================] - 15s 9ms/step - loss: 1.7508e-05 - mean_squared_error: 9.1212e-05 - acc: 0.9657 - val_loss: 3.2707e-05 - val_mean_squared_error: 1.5403e-04 - val_acc: 0.9028
Epoch 154/500
1605/1605 [==============================] - 16s 10ms/step - loss: 1.7504e-05 - mean_squared_error: 9.1195e-05 - acc: 0.9657 - val_loss: 3.2703e-05 - val_mean_squared_error: 1.5402e-04 - val_acc: 0.9028
Epoch 155/500
1605/1605 [==============================] - 14s 9ms/step - loss: 1.7502e-05 - mean_squared_error: 9.1186e-05 - acc: 0.9657 - val_loss: 3.2701e-05 - val_mean_squared_error: 1.5400e-04 - val_acc: 0.9028
Epoch 156/500
1605/1605 [==============================] - 15s 10ms/step - loss: 1.7499e-05 - mean_squared_error: 9.1170e-05 - acc: 0.9657 - val_loss: 3.2702e-05 - val_mean_squared_error: 1.5399e-04 - val_acc: 0.9009
Epoch 157/500
1605/1605 [==============================] - 15s 10ms/step - loss: 1.7497e-05 - mean_squared_error: 9.1168e-05 - acc: 0.

...

Epoch 343/500
1605/1605 [==============================] - 14s 9ms/step - loss: 1.7119e-05 - mean_squared_error: 8.9289e-05 - acc: 0.9645 - val_loss: 3.2593e-05 - val_mean_squared_error: 1.5241e-04 - val_acc: 0.9028
Epoch 344/500
1605/1605 [==============================] - 14s 9ms/step - loss: 1.7118e-05 - mean_squared_error: 8.9264e-05 - acc: 0.9645 - val_loss: 3.2587e-05 - val_mean_squared_error: 1.5237e-04 - val_acc: 0.9028
Epoch 345/500
1605/1605 [==============================] - 18s 11ms/step - loss: 1.7116e-05 - mean_squared_error: 8.9256e-05 - acc: 0.9645 - val_loss: 3.2580e-05 - val_mean_squared_error: 1.5236e-04 - val_acc: 0.9028
Epoch 346/500
1605/1605 [==============================] - 20s 12ms/step - loss: 1.7115e-05 - mean_squared_error: 8.9243e-05 - acc: 0.9645 - val_loss: 3.2581e-05 - val_mean_squared_error: 1.5233e-04 - val_acc: 0.9028
Epoch 347/500
1605/1605 [==============================] - 22s 13ms/step - loss: 1.7113e-05 - mean_squared_error: 8.9228e-05 - acc: 0.

Epoch 381/500
1605/1605 [==============================] - 14s 9ms/step - loss: 1.7056e-05 - mean_squared_error: 8.8960e-05 - acc: 0.9645 - val_loss: 3.2547e-05 - val_mean_squared_error: 1.5211e-04 - val_acc: 0.9028
Epoch 382/500
1605/1605 [==============================] - 13s 8ms/step - loss: 1.7054e-05 - mean_squared_error: 8.8954e-05 - acc: 0.9645 - val_loss: 3.2549e-05 - val_mean_squared_error: 1.5211e-04 - val_acc: 0.9028
Epoch 383/500
1605/1605 [==============================] - 14s 9ms/step - loss: 1.7052e-05 - mean_squared_error: 8.8944e-05 - acc: 0.9645 - val_loss: 3.2548e-05 - val_mean_squared_error: 1.5210e-04 - val_acc: 0.9047
Epoch 384/500
1605/1605 [==============================] - 13s 8ms/step - loss: 1.7050e-05 - mean_squared_error: 8.8921e-05 - acc: 0.9645 - val_loss: 3.2544e-05 - val_mean_squared_error: 1.5206e-04 - val_acc: 0.9047
Epoch 385/500
1605/1605 [==============================] - 14s 9ms/step - loss: 1.7049e-05 - mean_squared_error: 8.8906e-05 - acc: 0.964

Epoch 419/500
1605/1605 [==============================] - 16s 10ms/step - loss: 1.6995e-05 - mean_squared_error: 8.8715e-05 - acc: 0.9651 - val_loss: 3.2508e-05 - val_mean_squared_error: 1.5189e-04 - val_acc: 0.9028
Epoch 420/500
1605/1605 [==============================] - 14s 9ms/step - loss: 1.6993e-05 - mean_squared_error: 8.8705e-05 - acc: 0.9651 - val_loss: 3.2506e-05 - val_mean_squared_error: 1.5189e-04 - val_acc: 0.9028
Epoch 421/500
1605/1605 [==============================] - 16s 10ms/step - loss: 1.6992e-05 - mean_squared_error: 8.8700e-05 - acc: 0.9651 - val_loss: 3.2508e-05 - val_mean_squared_error: 1.5188e-04 - val_acc: 0.9028
Epoch 422/500
1605/1605 [==============================] - 14s 9ms/step - loss: 1.6990e-05 - mean_squared_error: 8.8702e-05 - acc: 0.9651 - val_loss: 3.2512e-05 - val_mean_squared_error: 1.5189e-04 - val_acc: 0.9028
Epoch 423/500
1605/1605 [==============================] - 14s 9ms/step - loss: 1.6988e-05 - mean_squared_error: 8.8705e-05 - acc: 0.9

Epoch 457/500
1605/1605 [==============================] - 15s 9ms/step - loss: 1.6937e-05 - mean_squared_error: 8.8420e-05 - acc: 0.9651 - val_loss: 3.2513e-05 - val_mean_squared_error: 1.5164e-04 - val_acc: 0.9028
Epoch 458/500
1605/1605 [==============================] - 16s 10ms/step - loss: 1.6935e-05 - mean_squared_error: 8.8420e-05 - acc: 0.9651 - val_loss: 3.2514e-05 - val_mean_squared_error: 1.5163e-04 - val_acc: 0.9028
Epoch 459/500
1605/1605 [==============================] - 17s 10ms/step - loss: 1.6934e-05 - mean_squared_error: 8.8416e-05 - acc: 0.9651 - val_loss: 3.2514e-05 - val_mean_squared_error: 1.5163e-04 - val_acc: 0.9028
Epoch 460/500
1605/1605 [==============================] - 17s 11ms/step - loss: 1.6935e-05 - mean_squared_error: 8.8408e-05 - acc: 0.9651 - val_loss: 3.2515e-05 - val_mean_squared_error: 1.5162e-04 - val_acc: 0.9028
Epoch 461/500
1605/1605 [==============================] - 14s 9ms/step - loss: 1.6933e-05 - mean_squared_error: 8.8406e-05 - acc: 0.

Epoch 495/500
1605/1605 [==============================] - 14s 9ms/step - loss: 1.6880e-05 - mean_squared_error: 8.8193e-05 - acc: 0.9657 - val_loss: 3.2504e-05 - val_mean_squared_error: 1.5145e-04 - val_acc: 0.9028
Epoch 496/500
1605/1605 [==============================] - 16s 10ms/step - loss: 1.6878e-05 - mean_squared_error: 8.8189e-05 - acc: 0.9657 - val_loss: 3.2501e-05 - val_mean_squared_error: 1.5145e-04 - val_acc: 0.9028
Epoch 497/500
1605/1605 [==============================] - 14s 9ms/step - loss: 1.6877e-05 - mean_squared_error: 8.8192e-05 - acc: 0.9651 - val_loss: 3.2495e-05 - val_mean_squared_error: 1.5144e-04 - val_acc: 0.9028
Epoch 498/500
1605/1605 [==============================] - 15s 9ms/step - loss: 1.6876e-05 - mean_squared_error: 8.8200e-05 - acc: 0.9645 - val_loss: 3.2501e-05 - val_mean_squared_error: 1.5147e-04 - val_acc: 0.9028
Epoch 499/500
1605/1605 [==============================] - 15s 10ms/step - loss: 1.6875e-05 - mean_squared_error: 8.8202e-05 - acc: 0.9